## 1. Analysis of costs to run animal shelters
<p>You are working as a data analyst in the animal welfare department. In preparation for next year's budget, the head of your department would like to know the total cost to shelter animals, broken down by each animal type and size combination (i.e., 'small dogs', 'large dogs', etc.). You need to prepare a table that contains this information.</p>
<p>The total cost to shelter an animal for the year is calculated as the sum of three factors: the size and animal type, the age, and the location.</p>
<ul>
<li>The base cost of sheltering an animal is based upon its size and type. The costs per animal type and size are contained in the <code>size_costs</code> table. The criteria for classifying size has recently been updated, and so you will need to categorize animals based upon the following table:</li>
</ul>
<table>
<thead>
<tr>
<th style="text-align:right;"></th>
<th style="text-align:right;">Small</th>
<th>Medium</th>
<th>Large</th>
</tr>
</thead>
<tbody>
<tr>
<td style="text-align:right;">Dog</td>
<td style="text-align:right;">&lt;= 10 lbs</td>
<td>10 lbs &lt; and &lt;= 30 lbs</td>
<td>30 lbs &lt;</td>
</tr>
<tr>
<td style="text-align:right;">Cat</td>
<td style="text-align:right;">&lt;= 5 lbs</td>
<td>5 lbs &lt; and &lt;= 7 lbs</td>
<td>7 lbs &lt;</td>
</tr>
<tr>
<td style="text-align:right;">Bird</td>
<td style="text-align:right;">&lt;= 0.7 lbs</td>
<td>0.7 lbs &lt; and &lt;= 1.1 lbs</td>
<td>1.1 lbs &lt;</td>
</tr>
</tbody>
</table>
<ul>
<li>Older animals cost more, and so an age cost (contained in the <code>age_costs</code> table) is added. Each animal's age should be calculated as the age by the end of the year (December 31st, 2021).</li>
<li>There is a location cost depending on where the animal is sheltered (contained in the <code>location_costs</code> table). </li>
<li>The calculation should not include animals that have been sponsored by private charities (sponsored animals are listed in the <code>sponsored_animals</code> table).</li>
</ul>
<p>For future visualization purposes, you will also need to include a <code>percentage</code> column in your result. This percentage should reflect the fraction of the total cost to be allocated to each animal and size combination.</p>
<p>The data you need is available in the tables shown in the database schema below.</p>
<h5 id="databaseschema">Database Schema</h5>
<p><img src="https://assets.datacamp.com/production/repositories/5934/datasets/a946a159c0024ee0995f7a030f2c0cf802203835/diagram.PNG" width="500" height="500"> </p>

In [1]:
%%sql
postgresql:///animal_shelters

'Connected: @animal_shelters'

In [147]:
animals = %sql select * from animals;
import pandas as pd
animals = pd.DataFrame(animals)
# Connect to database
from sqlalchemy import create_engine
engine = create_engine("postgresql:///animal_shelters");

# Query database
result = engine.execute("SELECT * FROM animals;")

# Change column in animals dataframe
animals.columns = list(result.keys())
animals.head()

 * postgresql:///animal_shelters
11899 rows affected.


,animalid,birthdate,animaltype,color,weight,location
0,80066a4b41ac06,05/11/2019,Cat,Blue Tabby/White,4.0,Fort Worth
1,968bf2e7230d2f,11/12/2012,Cat,Torbie,7.0,Houston
2,8aacfe19c56b0f,07/08/2013,Dog,White/Tan,45.0,Austin
3,7126d96889803f,04/05/2015,Cat,Black,3.0,Houston
4,9617240ad5b268,04/29/2014,Cat,Black,9.0,Houston


In [148]:
def size_converter(animal, weight):
    if animal == 'Dog':
        if weight <= 10:
            return 'Small'
        elif 10<weight<=30:
            return 'Medium'
        else:
            return 'Large'
    elif animal == 'Cat':
        if weight <= 5:
            return 'Small'
        elif 5<weight<=7:
            return 'Medium'
        else:
            return 'Large'
    elif animal == 'Bird':
        if weight <= 0.7:
            return 'Small'
        elif 0.7<weight<=1.1:
            return 'Medium'
        else:
            return 'Large'

In [149]:
animals['size'] = animals.apply(lambda x: size_converter(x['animaltype'], x['weight']), axis=1)

In [150]:
animals.head()

,animalid,birthdate,animaltype,color,weight,location,size
0,80066a4b41ac06,05/11/2019,Cat,Blue Tabby/White,4.0,Fort Worth,Small
1,968bf2e7230d2f,11/12/2012,Cat,Torbie,7.0,Houston,Medium
2,8aacfe19c56b0f,07/08/2013,Dog,White/Tan,45.0,Austin,Large
3,7126d96889803f,04/05/2015,Cat,Black,3.0,Houston,Small
4,9617240ad5b268,04/29/2014,Cat,Black,9.0,Houston,Large


In [151]:
animals['birthdate'] = pd.to_datetime(animals['birthdate'])

In [152]:
animals.head()

,animalid,birthdate,animaltype,color,weight,location,size
0,80066a4b41ac06,2019-05-11,Cat,Blue Tabby/White,4.0,Fort Worth,Small
1,968bf2e7230d2f,2012-11-12,Cat,Torbie,7.0,Houston,Medium
2,8aacfe19c56b0f,2013-07-08,Dog,White/Tan,45.0,Austin,Large
3,7126d96889803f,2015-04-05,Cat,Black,3.0,Houston,Small
4,9617240ad5b268,2014-04-29,Cat,Black,9.0,Houston,Large


In [153]:
last = pd.to_datetime('12/31/2021')
animals['age'] = (last-animals['birthdate']).astype('<m8[Y]')

In [154]:
animals.drop('birthdate',axis=1,inplace=True)

In [155]:
animals.head()

,animalid,animaltype,color,weight,location,size,age
0,80066a4b41ac06,Cat,Blue Tabby/White,4.0,Fort Worth,Small,2.0
1,968bf2e7230d2f,Cat,Torbie,7.0,Houston,Medium,9.0
2,8aacfe19c56b0f,Dog,White/Tan,45.0,Austin,Large,8.0
3,7126d96889803f,Cat,Black,3.0,Houston,Small,6.0
4,9617240ad5b268,Cat,Black,9.0,Houston,Large,7.0


In [156]:
sponsored = %sql select * from sponsored_animals;
import pandas as pd
sponsored = pd.DataFrame(sponsored)
# Connect to database
from sqlalchemy import create_engine
engine = create_engine("postgresql:///animal_shelters");

# Query database
result = engine.execute("SELECT * FROM sponsored_animals;")

# Change column in animals dataframe
sponsored.columns = list(result.keys())
sponsored.head()

 * postgresql:///animal_shelters
2142 rows affected.


,sponsorid,animaltype,location
0,7e18c2a067e4fd,Cat,Houston
1,32372ddbe6e3ec,Cat,Austin
2,3b5b82a5555895,Dog,Austin
3,da29bb3c402c1c,Dog,Austin
4,15bf11c3efa431,Cat,Austin


In [157]:
sponsor_id = list(sponsored['sponsorid'])
sponsor_id

['7e18c2a067e4fd',
 '32372ddbe6e3ec',
 '3b5b82a5555895',
 'da29bb3c402c1c',
 '15bf11c3efa431',
 '36806649e5722d',
 'd7062f851d9101',
 'd231eaa79ced6f',
 '18596d0453f828',
 'b3b7ec96b85208',
 '34c078327cb023',
 '457b274c411d80',
 'da0121755df78f',
 '6882528aed2a85',
 '62509d57146db2',
 'eeb87b8daa8bfd',
 'c8c655aed009c6',
 '835b07d7237ec4',
 '6fb51d036d6f79',
 '7e254a95bb00ac',
 '089605af08a8a4',
 'df19506168ad7e',
 '0918714dc3a6e1',
 'f4eb90ec7acc25',
 '64cc3ddbe2f5be',
 '214f8f6b359a1e',
 '7525712d71930c',
 '0ea863dbf1123f',
 '7c0c3e04e6a4d4',
 'ce9bc54c423ab3',
 'd70e8a48b1165f',
 '90ff982d8c9890',
 '01b6df09374aad',
 'a3ca1878263065',
 '5c0434772d6af1',
 '83066ff8f3b049',
 '60d76a3514e071',
 'b91a9885287008',
 'a391adcb8c4379',
 'e59fe601ac9d1e',
 'bc198be65dde75',
 '8f4d1c3844bb5c',
 'e35dd271530738',
 '7a72b8291dcc5e',
 'b45f5e3926570a',
 '4f685a2835db15',
 'fc726065526a28',
 'a5bc174a2acc31',
 '5025abc4977311',
 'e3e40530c8efb4',
 'b1eeaab6d18555',
 '28a3c319470d24',
 '04d9fe8f2b

In [158]:
#Exclude sponsored animals
animals = animals[~animals['animalid'].isin(sponsor_id)]

In [159]:
location = %sql select * from location_costs;
import pandas as pd
location = pd.DataFrame(location)
# Connect to database
from sqlalchemy import create_engine
engine = create_engine("postgresql:///animal_shelters");

# Query database
result = engine.execute("SELECT * FROM location_costs;")

# Change column in animals dataframe
location.columns = list(result.keys())
location.head()

 * postgresql:///animal_shelters
5 rows affected.


,location,costs
0,San Antonio,120
1,Dallas,110
2,Houston,140
3,Austin,135
4,Fort Worth,100


In [160]:
location_cost = pd.merge(animals,location,on='location')

In [161]:
location_cost.head()

,animalid,animaltype,color,weight,location,size,age,costs
0,968bf2e7230d2f,Cat,Torbie,7.0,Houston,Medium,9.0,140
1,9617240ad5b268,Cat,Black,9.0,Houston,Large,7.0,140
2,e03a24b436f6db,Dog,Yellow,13.0,Houston,Medium,8.0,140
3,fd8befa64f05ff,Cat,Brown Tabby,9.0,Houston,Large,4.0,140
4,9c6cbeb11aaabc,Dog,White/Black,22.0,Houston,Medium,14.0,140


In [162]:
location_cost.rename(columns={'costs':'loc_costs'}, inplace=True)

In [163]:
location_cost.head()

,animalid,animaltype,color,weight,location,size,age,loc_costs
0,968bf2e7230d2f,Cat,Torbie,7.0,Houston,Medium,9.0,140
1,9617240ad5b268,Cat,Black,9.0,Houston,Large,7.0,140
2,e03a24b436f6db,Dog,Yellow,13.0,Houston,Medium,8.0,140
3,fd8befa64f05ff,Cat,Brown Tabby,9.0,Houston,Large,4.0,140
4,9c6cbeb11aaabc,Dog,White/Black,22.0,Houston,Medium,14.0,140


In [164]:
age = %sql select * from age_costs;
import pandas as pd
age = pd.DataFrame(age)
# Connect to database
from sqlalchemy import create_engine
engine = create_engine("postgresql:///animal_shelters");

# Query database
result = engine.execute("SELECT * FROM age_costs;")

# Change column in animals dataframe
age.columns = list(result.keys())
age.head()

 * postgresql:///animal_shelters
51 rows affected.


,age,costs
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0


In [165]:
age['age'] = age['age'].astype('float')

In [166]:
age.head()

,age,costs
0,0.0,0
1,1.0,0
2,2.0,0
3,3.0,0
4,4.0,0


In [167]:
age_loc_cost = pd.merge(location_cost,age,on='age')

In [168]:
age_loc_cost.head()

,animalid,animaltype,color,weight,location,size,age,loc_costs,costs
0,968bf2e7230d2f,Cat,Torbie,7.0,Houston,Medium,9.0,140,150
1,346f914fbe622f,Cat,Brown Tabby,6.0,Houston,Medium,9.0,140,150
2,a1a4159214c0e9,Dog,Blue Merle/White,47.0,Houston,Large,9.0,140,150
3,da6dbbe378abc8,Dog,Tan/Cream,32.0,Houston,Large,9.0,140,150
4,bec6392753c2cb,Dog,Tan/White,10.0,Houston,Small,9.0,140,150


In [169]:
age_loc_cost.rename(columns={'costs':'age_costs'}, inplace=True)

In [170]:
age_loc_cost.head()

,animalid,animaltype,color,weight,location,size,age,loc_costs,age_costs
0,968bf2e7230d2f,Cat,Torbie,7.0,Houston,Medium,9.0,140,150
1,346f914fbe622f,Cat,Brown Tabby,6.0,Houston,Medium,9.0,140,150
2,a1a4159214c0e9,Dog,Blue Merle/White,47.0,Houston,Large,9.0,140,150
3,da6dbbe378abc8,Dog,Tan/Cream,32.0,Houston,Large,9.0,140,150
4,bec6392753c2cb,Dog,Tan/White,10.0,Houston,Small,9.0,140,150


In [171]:
size = %sql select * from size_costs;
import pandas as pd
size = pd.DataFrame(size)
# Connect to database
from sqlalchemy import create_engine
engine = create_engine("postgresql:///animal_shelters");

# Query database
result = engine.execute("SELECT * FROM size_costs;")

# Change column in animals dataframe
size.columns = list(result.keys())
size.head()

 * postgresql:///animal_shelters
9 rows affected.


,sizeid,animaltype,size,costs
0,DL,Dog,Large,175
1,DM,Dog,Medium,150
2,DS,Dog,Small,125
3,CL,Cat,Large,140
4,CM,Cat,Medium,120


In [172]:
total_cost = pd.merge(age_loc_cost,size,on=['animaltype', 'size'])

In [173]:
total_cost.head()

,animalid,animaltype,color,weight,location,size,age,loc_costs,age_costs,sizeid,costs
0,968bf2e7230d2f,Cat,Torbie,7.0,Houston,Medium,9.0,140,150,CM,120
1,346f914fbe622f,Cat,Brown Tabby,6.0,Houston,Medium,9.0,140,150,CM,120
2,0224100f50a776,Cat,Black,6.0,Houston,Medium,9.0,140,150,CM,120
3,27c67087ac1181,Cat,Brown Tabby,7.0,Houston,Medium,9.0,140,150,CM,120
4,60bf447f0efdd3,Cat,Tortie,7.0,Houston,Medium,9.0,140,150,CM,120


In [174]:
total_cost['total'] = total_cost['loc_costs']+total_cost['age_costs']+total_cost['costs']

In [175]:
total_cost.head()

,animalid,animaltype,color,weight,location,size,age,loc_costs,age_costs,sizeid,costs,total
0,968bf2e7230d2f,Cat,Torbie,7.0,Houston,Medium,9.0,140,150,CM,120,410
1,346f914fbe622f,Cat,Brown Tabby,6.0,Houston,Medium,9.0,140,150,CM,120,410
2,0224100f50a776,Cat,Black,6.0,Houston,Medium,9.0,140,150,CM,120,410
3,27c67087ac1181,Cat,Brown Tabby,7.0,Houston,Medium,9.0,140,150,CM,120,410
4,60bf447f0efdd3,Cat,Tortie,7.0,Houston,Medium,9.0,140,150,CM,120,410


In [176]:
total_cost = total_cost[['animaltype','size','total']]

In [177]:
total_cost['animaltype'] = total_cost['animaltype'].str.lower()
total_cost['size'] = total_cost['size'].str.lower()

In [178]:
total_cost.head()

,animaltype,size,total
0,cat,medium,410
1,cat,medium,410
2,cat,medium,410
3,cat,medium,410
4,cat,medium,410


In [179]:
total_cost.groupby(by=['animaltype','size']).sum()

total
animaltype size          
bird       large     7770
           medium    3460
           small     1615
cat        large   439390
           medium  250575
           small   518015
dog        large   977565
           medium  941845
           small   336530

In [180]:
type = ['bird','bird','bird','cat','cat','cat','dog','dog','dog']
size = ['small','medium','large','small','medium','large','small','medium','large']
total = [1615,3460,7770,518015,250575,439390,336530,941845,977565]

In [181]:
data = {'animaltype':type, 'size':size, 'total':total}
data = pd.DataFrame(data)
data

,animaltype,size,total
0,bird,small,1615
1,bird,medium,3460
2,bird,large,7770
3,cat,small,518015
4,cat,medium,250575
5,cat,large,439390
6,dog,small,336530
7,dog,medium,941845
8,dog,large,977565


In [182]:
grandtotal = data['total'].sum()
grandtotal

3476765

In [183]:
data['percentage'] = round(data['total']/grandtotal*100,2)

In [184]:
data

,animaltype,size,total,percentage
0,bird,small,1615,0.05
1,bird,medium,3460,0.10
2,bird,large,7770,0.22
3,cat,small,518015,14.90
4,cat,medium,250575,7.21
5,cat,large,439390,12.64
6,dog,small,336530,9.68
7,dog,medium,941845,27.09
8,dog,large,977565,28.12
